# Scrape BeerAdvocate for beer ratings
This notebook scrapes BeerAdvocate for ratings of beer to subsequently perform linear regression on the ratings in terms of other available information on the beer. We start with the style page since we can use this to obtain a list of all beers of that style easily from there.

In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import os
import re
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [14]:
start_url = "https://www.beeradvocate.com/beer/style/"

styles = os.path.join(os.path.curdir, "data", "styles.html")

if not os.path.exists(styles):
    os.makedirs("data")
    r = requests.get(start_url)
    page = r.text
    with open(styles, 'w') as file:
        file.write(page)
else:
    with open(styles, 'r') as file:
        page = file.read()

In [15]:
soup = BeautifulSoup(page,"lxml")

In [19]:
soup.find('table')

<table border="0" cellpadding="0" cellspacing="0" class="break" width="100%">
<tr>
<td align="left" valign="top" width="50%">
<table border="0" cellpadding="2" cellspacing="0" width="100%">
<tr>
<td align="left" bgcolor="#000000" valign="top"><span style="color: #FFFFFF; font-weight: bold; font-size: 8pt;">Ale Styles</span></td>
<td align="left" class="hr_bottom_black" valign="top" width="100"> </td>
</tr><tr>
<td align="left" colspan="2" valign="top">
<br/><b>American Ales</b><br/><a href="/beer/style/128/">American Amber / Red Ale</a><br/><a href="/beer/style/19/">American Barleywine</a><br/><a href="/beer/style/175/">American Black Ale</a><br/><a href="/beer/style/99/">American Blonde Ale</a><br/><a href="/beer/style/73/">American Brown Ale</a><br/><a href="/beer/style/94/">American Dark Wheat Ale</a><br/><a href="/beer/style/140/">American Double / Imperial IPA</a><br/><a href="/beer/style/157/">American Double / Imperial Stout</a><br/><a href="/beer/style/116/">American IPA</a><br